# v2 exploration trying to make it work better

In [ ]:
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
# Local modules
sys.path.append('..')
import reproducibility
import pandas as pd
from utils import print_dict_summary
from data_funcs import load_and_fix_data, rmse
from moisture_rnn import RNN, RNN_LSTM, create_rnn_data2
from moisture_rnn_pkl import pkl2train
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import mean_squared_error
from utils import hash2
import copy
import logging
from utils import logging_setup

In [ ]:
logging_setup()

## Test Data Creation

In [ ]:
import yaml

with open("params.yaml") as file:
    params = yaml.safe_load(file)["rnn"]
# params.update({'scale': 1})

In [ ]:
import importlib
import moisture_rnn_pkl
importlib.reload(moisture_rnn_pkl)
from moisture_rnn_pkl import pkl2train

In [ ]:
# params.update({'features_list': ['Ed', 'rain', 'solar', 'wind']})
train = pkl2train(['data/reproducibility_dict2.pickle', "data/test_CA_202401.pkl"])
# train = pd.read_pickle('train.pkl')

In [ ]:
list(train.keys())[0:5]

In [ ]:
print(train['reproducibility']['features_list'])
print(train['reproducibility']['X'].shape)

In [ ]:
print(train['CRVC1_202401']['features_list'])
print(train['CRVC1_202401']['X'].shape)

In [ ]:
import importlib
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import create_rnn_data2

In [ ]:
params.update({'val_frac': .2, 'scale': True, 'scaler': 'standard'})
rnn_dat = create_rnn_data2(train['CRVC1_202401'], params)

In [ ]:
print(rnn_dat['X'].shape)
print(rnn_dat['features_list'])

In [ ]:
params.update({'features_list': ['wind', 'Ed', 'Ew', 'solar', 'lat', 'rain']})
rnn_dat = create_rnn_data2(train['CRVC1_202401'], params)

In [ ]:
print(rnn_dat['X'].shape)
print(rnn_dat['features_list'])

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)
m, errs = rnn.run_model(rnn_dat)

In [ ]:
params.update({'scale': False})
rnn_dat = create_rnn_data2(train['CRVC1_202401'], params)
print(rnn_dat['X_train'][0,:])
reproducibility.set_seed()
rnn = RNN(params)
m, errs = rnn.run_model(rnn_dat)

In [ ]:
errs

## LSTM

In [ ]:
import importlib 
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNN_LSTM

In [ ]:
with open("params.yaml") as file:
    params = yaml.safe_load(file)["lstm"]
    
rnn_dat = create_rnn_data2(train['CRVC1_202401'],params)

In [ ]:
params

In [ ]:
params.update({'epochs': 200})
reproducibility.set_seed()
lstm = RNN_LSTM(params)
m, errs = lstm.run_model(rnn_dat)